In [9]:
import pronouncing as p
import nltk as n
from typing import List

In [2]:
p.phones_for_word("snappiest")

['S N AE1 P IH0 EH2 S T']

In [4]:
# n.download()

In [5]:
from nltk.corpus import gutenberg

In [6]:
gutenberg.words()

['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']', ...]

In [7]:
len(gutenberg.words())

2621613

In [8]:
gutenberg.words()[0:10]

['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']', 'VOLUME', 'I', 'CHAPTER']

In [22]:
# stress_string = "0101010101"
# markov model

# need a function to generate possible next words
def get_next_word(current_word, model):
    """
        :type current_word: string
        :type mode: markov_model
        :rtype: List[string]
    """
    words = model.get_next_word(current_word)
    return words

In [38]:
def make_one_zero_str(length: int, start_stress: int) -> List[str]:
    one_zero_str = ""
    for index in range(start_stress, length + start_stress):
        one_zero_str += str(index % 2)
        
    return one_zero_str

def get_possible_stresses(current_stresses: str) -> List[str]:
    """
    can assume that current_stresses is 0[1,2] repeated
    """
    syllables = len(current_stresses)
    remaining_stresses = 10 - syllables
    start_stress = remaining_stresses % 2  # if even, start with 0
    possible_stress_lengths = list(range(1, remaining_stresses + 1))
    return list(map(lambda length: make_one_zero_str(length, start_stress), possible_stress_lengths))
    
    
    
get_possible_stresses("010101")
    

['0', '01', '010', '0101']

In [39]:
def word_matches_stress(word: str, stress_pattern_match: str) -> bool:
    pronunciations = p.phones_for_word(word)
    for pronunciation in pronunciations:
        stress_pattern = p.stresses(pronunciation)
        if compare_stress_pattern(stress_pattern, stress_pattern_match):
            return True
    return False
        
    
def compare_stress_pattern(stress_pattern_1: str, stress_pattern_2: str) -> bool:
    if len(stress_pattern_1) != len(stress_pattern_2):
        return False
    else:
        return all(int(x) + int(y) == 0 or int(x) + int(y) >=2 for (x, y) in zip(stress_pattern_1, stress_pattern_2))
        
        
assert compare_stress_pattern("102", "201")
assert compare_stress_pattern("", "")
assert compare_stress_pattern("10201010", "20101020")
assert word_matches_stress("adam", "10")
assert word_matches_stress("janice", "20")
assert word_matches_stress("compare", "01")

In [40]:
def filter_possible_stresses(possible_stresses: List[str], words: List[str]) -> List[str]:
    """
        :type possible_stresses List[stress_strings]
        :type words List[strings]
        :filtered_words List[strings]
    """
    filtered_words = []
    for stress_pattern in possible_stresses:
        for word in words:
            if word_matches_stress(word, stress_pattern):
                filtered_words += word
    return filtered_words